In [7]:
import sys
import re
import json
import _pickle as p
import pandas as pd
import numpy as np
from importlib import reload
from IPython.display import display, HTML

sys.path.append('..')
sys.path.append('../lib')
import stats as S
S = reload(S)

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
display(HTML("<style>.container{width: 100%}</style>"))

/home/aynur/Installed/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  app.launch_new_instance()


In [9]:
lemm = S.lemm_h

In [ ]:
articles = pd.read_csv('medical_site.csv')
pharma = pd.read_csv('pharma.csv')
pharma.drop(['web-scraper-order', 'web-scraper-start-url', 'Page', 'Page-href', 'PharmaLink', 'AnalogsLinks-href', 'Lots', 'Instruction'], axis=1, inplace=True)

txt = "\n".join(articles['ArticleText'].map(str)) + "\n" + "\n".join(pharma['PharmacologicAction'].map(str))
txt_lemm = lemm(txt)
kwds = S.keywords(txt_lemm)
word_cnt = len(kwds)
word_cnt_log_inv = 1 / np.log2(word_cnt)
display(word_cnt, word_cnt_log_inv)
kwds_sorted = sorted(kwds.items(), key=lambda x: -x[1])
kwds_inverted_ph = {i: word_cnt_log_inv/v for i, v in kwds.items()}
v = kwds_inverted_ph.values()
with open('word_importance.ph.json', 'w') as f:
    f.write(json.dumps(kwds_inverted_ph))
display(min(v), max(v))

Ph = pharma.loc[pharma['PharmacologicAction'].notnull()]
Ph.loc[:, 'Lemm'] = Ph.loc[:, 'PharmacologicAction'].apply(lemm)
Ph.loc[:, 'Kw'] = Ph.loc[:, 'Lemm'].apply(lambda x: {i: 1 for i, v in S.keywords(x).items()})
Ph.loc[:, 'KwMult'] = Ph.loc[:, 'Kw'].apply(lambda x: S.multiply2stats(kwds_inverted_ph, x, max_dist=0))


Ph.drop(['Lemm', 'Kw'], axis=1, inplace=True)
Ph.to_csv('pharma_kw.csv')

In [3]:
pattern = re.compile('[^\w\d\s]+') 
Sdict = pd.read_csv('med_terms.csv')
Sdict.drop(['web-scraper-order', 'web-scraper-start-url', 'Pages', 'Pages-href'], axis=1, inplace=True)

txt = "\n".join(Sdict['Definitions'].map(str))
txt_lemm = lemm(txt)
kwds = S.keywords(txt_lemm)
word_cnt = len(kwds)
word_cnt_log_inv = 1 / np.log2(word_cnt)
display(word_cnt, word_cnt_log_inv)
kwds_sorted = sorted(kwds.items(), key=lambda x: -x[1])
kwds_inverted_def = {i: word_cnt_log_inv/v for i, v in kwds.items()}
v = kwds_inverted_def.values()
with open('word_importance.def.json', 'w') as f:
    f.write(json.dumps(kwds_inverted_def))
min(v), max(v)

Sdict.loc[:, 'Term'] = Sdict.loc[:, 'Definitions'].apply(lambda x: pattern.split(x, 1)[0])
Sdict.loc[:, 'Definitions'] = Sdict.loc[:, 'Definitions'].apply(lambda x: pattern.split(x, 1)[1])
Sdict[['TermLemm', 'DefLemm']] = Sdict.loc[:, ['Term', 'Definitions']].applymap(S.lemm_m)
Sdict.loc[:, 'DefKw'] = Sdict.loc[:, 'DefLemm'].apply(lambda x: S.keywords(x))
# Sdict.loc[:, 'TermKw'] = Sdict.loc[:, 'TermLemm'].apply(lambda x: S.keywords(x))
Sdict.loc[:, 'KwMult'] = Sdict.loc[:, 'DefKw'].apply(lambda x: S.multiply2stats(kwds_inverted_def, x, max_dist=0))
# Sdict.loc[:, 'DefKwFilt'] = Sdict.loc[:, 'DefKwMult'].apply(lambda x: )
Sdict.drop(['DefKw', 'DefLemm'], axis=1, inplace=True)
Sdict.to_csv('med_terms_kw.csv')

3905

0.08381452064712568

In [4]:
Pdict = pd.read_csv('medical_profs.csv')
Pdict.drop(['web-scraper-order', 'web-scraper-start-url', 'ProfLinks-href'], axis=1, inplace=True)

txt = "\n".join(Pdict['DescriptionText'].map(str))
txt_lemm = S.lemm_m(txt)
kwds = S.keywords(txt_lemm)
word_cnt = len(kwds)
word_cnt_log_inv = 1 / np.log2(word_cnt)
display(word_cnt, word_cnt_log_inv)
kwds_sorted = sorted(kwds.items(), key=lambda x: -x[1])
kwds_inverted_prof = {i: word_cnt_log_inv/v for i, v in kwds.items()}
v = kwds_inverted_prof.values()
with open('word_importance.def.json', 'w') as f:
    f.write(json.dumps(kwds_inverted_prof))
min(v), max(v)

Pdict[['ProfLemm', 'DescLemm']] = Pdict.loc[:, ['ProfLinks', 'DescriptionText']].applymap(S.lemm_m)
Pdict.loc[:, 'DescKw'] = Pdict.loc[:, 'DescLemm'].apply(lambda x: S.keywords(x))
Pdict.loc[:, 'KwMult'] = Pdict.loc[:, 'DescKw'].apply(lambda x: S.multiply2stats(kwds_inverted_prof, x, max_dist=0))
Pdict.drop(['DescKw', 'DescLemm', 'ProfLemm'], axis=1, inplace=True)
Pdict.drop([1, 3, 4, 5, 11, 16, 26, 27, 33], axis=0, inplace=True)
Pdict.to_csv('medical_profs_kw.csv')

573

0.10914181288692824

In [5]:
for ds, THRESHOLD, fname_basis in [
    (Ph, 1e-2, 'pharma_kw_thr'),
    (Pdict, 1e-2, 'medical_profs_kw_thr'),
    (Sdict, 1e-2, 'med_terms_kw_thr'),
]:
    ds.loc[:, 'KwFilt'] = ds.loc[:, 'KwMult'].map(lambda x: {i: v for i, v in x.items() if v > max(x.values())*THRESHOLD})
    ds.loc[:, 'FSum'] = ds.loc[:, 'KwFilt'].map(lambda x: sum(x.values()))
    ds.loc[:, 'KwNorm'] = ds.apply(lambda x: {i: v/x['FSum'] for i, v in x['KwFilt'].items()}, axis=1)
    ds.drop(['KwFilt', 'FSum'], axis=1, inplace=True)
    ds.to_csv(f'{fname_basis}{THRESHOLD:.6f}.csv')

In [6]:
display(Ph.head())
display(Sdict.head())
display(Pdict.head())

,PharmaLink-href,PharmaName,ActiveSubstance,PharmacologicAction,AnaloguesText,AnalogsLinks,Lots-href,Аптека/Адрес,Метро,Цена,KwMult,KwNorm
0,https://www.rlsnet.ru/tn_index_id_3684.htm,Энап®-Н (Enap®-H),Гидрохлоротиазид + Эналаприл (Hydrochlorothiazide + Enalapril),"Фармакологическое действие — диуретическое, гипотензивное",NaN,NaN,NaN,NaN,NaN,NaN,"{'действ': 3.52138505444121e-06, 'фармакологическ': 3.6552638821525668e-06, 'гипотензивн': 2.935682247676222e-05, 'диуретическ': 9.601024604744148e-05}","{'действ': 0.026567725139312864, 'фармакологическ': 0.02757779811958033, 'гипотензивн': 0.22148784596633259, 'диуретическ': 0.7243666307747743}"
1,https://www.rlsnet.ru/tn_index_id_29955.htm,Энзикс дуо форте (Enzix duo forte),Индапамид + Эналаприл (Indapamide + Enalapril),Фармакологическое действие — гипотензивное,NaN,NaN,NaN,NaN,NaN,NaN,"{'действ': 3.52138505444121e-06, 'фармакологическ': 3.6552638821525668e-06, 'гипотензивн': 2.935682247676222e-05}","{'действ': 0.09638791273346813, 'фармакологическ': 0.10005246533501513, 'гипотензивн': 0.8035596219315168}"
2,https://www.rlsnet.ru/tn_index_id_29592.htm,Элефлокс (Eleflox®),Левофлоксацин* (Levofloxacin),"Фармакологическое действие — бактерицидное, антибактериальное широкого спектра",NaN,NaN,NaN,NaN,NaN,NaN,"{'антибактериальн': 4.3951357079495434e-05, 'широк': 5.773426805688516e-05, 'действ': 3.52138505444121e-06, 'спектр': 6.040435200715996e-05, 'фармакологическ': 3.6552638821525668e-06, 'бактерицидн': 0.00010666161386780479}","{'антибактериальн': 0.15928546163954793, 'широк': 0.20923653217872226, 'действ': 0.012761959613505342, 'спектр': 0.21891326534231065, 'фармакологическ': 0.01324715398047777, 'бактерицидн': 0.3865556272454361}"
3,https://www.rlsnet.ru/tn_index_id_3682.htm,Энап® (Enap®),Эналаприл* (Enalapril*),Фармакологическое действие — гипотензивное,NaN,NaN,NaN,NaN,NaN,NaN,"{'действ': 3.52138505444121e-06, 'фармакологическ': 3.6552638821525668e-06, 'гипотензивн': 2.935682247676222e-05}","{'действ': 0.09638791273346813, 'фармакологическ': 0.10005246533501513, 'гипотензивн': 0.8035596219315168}"
4,https://www.rlsnet.ru/tn_index_id_66219.htm,Эдарби® (Edarbi),Азилсартана медоксомил* (Azilsartan medoxomil*),"Фармакологическое действие — гипотензивное, блокирующее АТ1-рецепторы ангиотензина II",NaN,NaN,NaN,NaN,NaN,NaN,"{'ii': 0.00017305846850051328, 'действ': 3.52138505444121e-06, 'рецептор': 0.0001433196426505286, 'фармакологическ': 3.6552638821525668e-06, 'гипотензивн': 2.935682247676222e-05, 'блокир': 0.00014886749978538775, 'ангиотензин': 0.00022402390744403014, 'ат1': 0.00022622022026210886}","{'ii': 0.18177967372281537, 'действ': 0.0036988437017563404, 'рецептор': 0.15054217285533364, 'фармакологическ': 0.003839469293965896, 'гипотензивн': 0.030836246602682842, 'блокир': 0.15636961180457234, 'ангиотензин': 0.23531349349231762, 'ат1': 0.237620488526556}"


,Definitions,Term,TermLemm,KwMult,KwNorm
0,"жидкая часть крови, в которой находятся форменные элементы (эритроциты, лейкоциты, тромбоциты). По изменениям в составе плазмы крови диагностируются различные заболевания (ревматизм, сахарный диабет и тд.). Из плазмы крови готовят лекарственные препараты.",Плазма,[плазм],"{'плазм': 0.01523900375402285, 'жидк': 0.008381452064712568, 'част': 0.0013303892166210425, 'кров': 0.001591414948996057, 'котор': 0.0008640672231662441, 'наход': 0.01047681508089071, 'формен': 0.02095363016178142, 'элемент': 0.00364410959335329, 'эритроцит': 0.004190726032356284, 'лейкоцит': 0.005587634709808378, 'тромбоцит': 0.013969086774520946, 'изменен': 0.005986751474794691, 'состав': 0.008381452064712568, 'диагностир': 0.08381452064712568, 'различн': 0.0023281811290868245, 'заболеван': 0.00044820599276537796, 'ревматизм': 0.013969086774520946, 'сахарн': 0.007619501877011425, 'диабет': 0.007619501877011425, 'тд': 0.005986751474794691, 'готов': 0.04190726032356284, 'лекарствен': 0.0029933757373973454, 'препарат': 0.0026192037702226774}","{'плазм': 0.056555139149813166, 'жидк': 0.031105326532397243, 'част': 0.004937353417840832, 'кров': 0.005906074658050109, 'котор': 0.0032067346940615714, 'наход': 0.03888165816549655, 'формен': 0.0777633163309931, 'элемент': 0.013524055014085757, 'эритроцит': 0.015552663266198622, 'лейкоцит': 0.020736884354931494, 'тромбоцит': 0.05184221088732874, 'изменен': 0.02221809038028374, 'состав': 0.031105326532397243, 'диагностир': 0.3110532653239724, 'различн': 0.008640368481221456, 'ревматизм': 0.05184221088732874, 'сахарн': 0.028277569574906583, 'диабет': 0.028277569574906583, 'тд': 0.02221809038028374, 'готов': 0.1555266326619862, 'лекарствен': 0.01110904519014187, 'препарат': 0.009720414541374138}"
1,"ограничение подвижности сустава, вызванное рубцовым стягиванием кожи, сухожилий, заболеваниями мышц, сустава, болевым рефлексом, др. причинами.",Контрактура,[контрактур],"{'заболеван': 0.00044820599276537796, 'ограничен': 0.013969086774520946, 'подвижн': 0.009312724516347298, 'сустав': 0.007982335299726254, 'вызва': 0.01047681508089071, 'рубцов': 0.08381452064712568, 'стягиван': 0.08381452064712568, 'кож': 0.0011973502949589382, 'сухожил': 0.01047681508089071, 'мышц': 0.0017105004213699118, 'болев': 0.027938173549041892, 'рефлекс': 0.011973502949589382, 'др': 0.0006497249662567882, 'причин': 0.002890155884383644}","{'ограничен': 0.052603068327227366, 'подвижн': 0.03506871221815158, 'сустав': 0.030058896186987066, 'вызва': 0.039452301245420526, 'рубцов': 0.3156184099633642, 'стягиван': 0.3156184099633642, 'кож': 0.004508834428048059, 'сухожил': 0.039452301245420526, 'мышц': 0.006441192040068658, 'болев': 0.10520613665445473, 'рефлекс': 0.0450883442804806, 'причин': 0.010883393447012559}"
2,"вызванное сокращением сердца сотрясение передней стенки грудной клетки, распространяющееся в подложечную область. Сотрясение в области верхушки сердца называется верхушечным толчком. Их смещение, расширение площади, изменение силы помогают при диагностике заболеваний сердца.",Сердечный толчок,"[сердечн, толчок]","{'изменен': 0.005986751474794691, 'заболеван': 0.00044820599276537796, 'вызва': 0.01047681508089071, 'сокращен': 0.003991167649863127, 'сердц': 0.005238407540445355, 'сотрясен': 0.08381452064712568, 'передн': 0.008381452064712568, 'стенк': 0.004930265920419157, 'грудн': 0.009312724516347298, 'клетк': 0.0010221283005747033, 'распространя': 0.027938173549041892, 'подложечн': 0.08381452064712568, 'област': 0.006984543387260473, 'верхушк': 0.08381452064712568, 'называ': 0.007619501877011425, 'верхушечн': 0.08381452064712568, 'толчк': 0.027938173549041892, 'смещен': 0.04190726032356284, 'расширен': 0.007619501877011425, 'площад': 0.08381452064712568, 'сил': 0.027938173549041892, 'помога': 0.027938173549041892, 'диагностик': 0.01047681508089071}","{'изменен': 0.00914325244734599, 'вызва': 0.016000691782855483, 'сокращен': 0.006095501631563993, 'сердц': 0.008000345891427741, 'сотрясен': 0.1280055

,ProfLinks,DescriptionText,KwMult,KwNorm
0,Диетолог,"Дието́лог — специалист в области диетологии, соответствующий академическим и профессиональным требованиям. Врач-диетолог является дипломированным врачом, закончившим медицинский вуз и получившим затем дополнительное образование по диетологии, подтвержденное соответствующим дипломом. Является экспертом в области питания. Врачи-диетологи занимаются лечением ожирения, а также дают рекомендации пациентам по вопросам правильного питания, ведения здорового образа жизни, либо ставят цели для достижения определённых результатов, связанных со здоровьем.","{'диет': 0.10914181288692824, 'лог': 0.015591687555275463, 'специалист': 0.006063434049273791, 'област': 0.024253736197095165, 'диетолог': 0.10914181288692824, 'соответств': 0.05457090644346412, 'академическ': 0.10914181288692824, 'профессиональн': 0.05457090644346412, 'требован': 0.10914181288692824, 'врач': 0.008849336180021209, 'явля': 0.02728545322173206, 'дипломирова': 0.10914181288692824, 'законч': 0.10914181288692824, 'медицинск': 0.0036380604295642746, 'вуз': 0.10914181288692824, 'получ': 0.015591687555275463, 'зат': 0.10914181288692824, 'дополнительн': 0.10914181288692824, 'образован': 0.006420106640407544, 'подтвержден': 0.10914181288692824, 'диплом': 0.05457090644346412, 'эксперт': 0.05457090644346412, 'питан': 0.10914181288692824, 'занима': 0.01364272661086603, 'лечен': 0.012126868098547582, 'ожирен': 0.10914181288692824, 'такж': 0.02182836257738565, 'дают': 0.10914181288692824, 'рекомендац': 0.10914181288692824, 'пациент': 0.015591687555275463, 'вопрос': 0.03638060429564275, 'правильн': 0.10914181288692824, 'веден': 0.10914181288692824, 'здоров': 0.0727612085912855, 'образ': 0.10914181288692824, 'жизн': 0.05457090644346412, 'либ': 0.05457090644346412, 'став': 0.10914181288692824, 'цел': 0.05457090644346412, 'достижен': 0.10914181288692824, 'определен': 0.03638060429564275, 'результат': 0.10914181288692824, 'связа': 0.05457090644346412}","{'диет': 0.037176059535169795, 'лог': 0.005310865647881399, 'специалист': 0.0020653366408427665, 'област': 0.008261346563371066, 'диетолог': 0.037176059535169795, 'соответств': 0.018588029767584897, 'академическ': 0.037176059535169795, 'профессиональн': 0.018588029767584897, 'требован': 0.037176059535169795, 'врач': 0.0030142750974462, 'явля': 0.009294014883792449, 'дипломирова': 0.037176059535169795, 'законч': 0.037176059535169795, 'медицинск': 0.0012392019845056598, 'вуз': 0.037176059535169795, 'получ': 0.005310865647881399, 'зат': 0.037176059535169795, 'дополнительн': 0.037176059535169795, 'образован': 0.0021868270314805765, 'подтвержден': 0.037176059535169795, 'диплом': 0.018588029767584897, 'эксперт': 0.018588029767584897, 'питан': 0.037176059535169795, 'занима': 0.004647007441896224, 'лечен': 0.004130673281685533, 'ожирен': 0.037176059535169795, 'такж': 0.00743521190703396, 'дают': 0.037176059535169795, 'рекомендац': 0.037176059535169795, 'пациент': 0.005310865647881399, 'вопрос': 0.0123920198450566, 'правильн': 0.037176059535169795, 'веден': 0.037176059535169795, 'здоров': 0.0247840396901132, 'образ': 0.037176059535169795, 'жизн': 0.018588029767584897, 'либ': 0.018588029767584897, 'став': 0.037176059535169795, 'цел': 0.018588029767584897, 'достижен': 0.037176059535169795, 'определен': 0.0123920198450566, 'результат': 0.037176059535169795, 'связа': 0.018588029767584897}"
2,Акушерка,"Акуше́рка (фр. accoucher — помогать при родах[1]) — средний медицинский работник, специалист в акушерстве, оказывающий помощь беременным и роженицам (во время родов). В России для работы акушеркой необходимо получить среднее специальное образование по специальности «Медико-акушерское дело». Квалификации: «акушер(ка)», «фельдшер-акушер». Не следует путать с врачом акушер-гинекологом (высшее медицинское образование).","{'специалист': 0.006063434049273791, 'врач': 0.0029497787266737365, 'медицинск': 0.007276120859128549, 'получ': 0.015591687555275463, 'образован': 0.012840213280815088, '1': 0.007276120859